
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [63]:
# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
import requests
import json

In [ ]:
from src.support import ine_datos_demograficos, ine_datos_economicos, cambiar_nombre

### Ejecución para todos los años

In [9]:
for anio in range(2019, 2022):

    # Scrapear datos demográficos
    ine_datos_demograficos(anio)
    cambiar_nombre(anio, 'Demograficos')
    # Scrapear datos económicos
    ine_datos_economicos(anio)
    cambiar_nombre(anio, 'Economicos')

### Ejecución de extracción de API

In [31]:
# Headers de la API
headers = {"Accept": 'application/json',
           "Content-Type": 'application/json',
           "Host": 'apidatos.ree.es'}

# Comunidades/ciudades autónomas
id_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [35]:
# Definimos la ruta donde guardaremos los datos
ruta_datos_demanda = '/Users/javi/Documents/Hackio/Laboratorios/Laboratorio-ETL-Extraccion/datos/demanda'

# Creamos el directorio en caso de que no exista
os.makedirs(ruta_datos_demanda, exist_ok= True)

# Iteramos por comunidad
for com_aut, geo_id in id_comunidades.items():

    # Iteramos por año
    for anio in range(2019, 2022):

        # Definimos la sub-ruta para cada año
        ruta_anios = os.path.join(ruta_datos_demanda, str(anio))
        # Creamos la carpeta en caso de que no exista
        os.makedirs(ruta_anios, exist_ok= True)
        # Llamamos a la API
        response = requests.get(f'https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={anio}-01-01T00:00&end_date={anio}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={geo_id}', headers=headers)
        
        if response.status_code == 200:

            # Si la petición es satisfactoria guardamos el resultado
            result = response.json()
            ruta_com_aut = os.path.join(ruta_anios, com_aut + '.json')
            
            with open(ruta_com_aut, "w") as archivo:
                json.dump(result, archivo)

        else:
            print(f"Ha ocurrido algún problema en {com_aut}, {anio}. Código {response.status_code}")

In [ ]:
# Definimos la ruta donde guardaremos los datos
ruta_datos_generacion = '/Users/javi/Documents/Hackio/Laboratorios/Laboratorio-ETL-Extraccion/datos/generacion'

# Creamos el directorio en caso de que no exista
os.makedirs(ruta_datos_generacion, exist_ok= True)

# Iteramos por comuni
for com_aut, geo_id in id_comunidades.items():

    for anio in range(2019, 2022):

        # Definimos la sub-ruta para cada año
        ruta_anios = os.path.join(ruta_datos_generacion, str(anio))
        # Creamos la carpeta en caso de que no exista
        os.makedirs(ruta_anios, exist_ok= True)
        # Llamamos a la API
        response = requests.get(f'https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={anio}-01-01T00:00&end_date={anio}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={geo_id}', headers=headers)
        
        if response.status_code == 200:

            # Si la petición es satisfactoria guardamos el resultado
            result = response.json()
            ruta_com_aut = os.path.join(ruta_anios, com_aut + '.json')
            
            with open(ruta_com_aut, "w") as archivo:
                json.dump(result, archivo)

        else:
            print(f"Ha ocurrido algún problema en {com_aut}, {anio}. Código {response.status_code}")